In [1]:
import pandas as pd
import numpy as np

In [2]:
import sqlite3
import pandas as pd

In [3]:
def pd_to_sqlDB(input_df: pd.DataFrame,
                table_name: str,
                db_name: str = 'default.db') -> None:

    '''Take a Pandas dataframe `input_df` and upload it to `table_name` SQLITE table
    Args:
        input_df (pd.DataFrame): Dataframe containing data to upload to SQLITE
        table_name (str): Name of the SQLITE table to upload to
        db_name (str, optional): Name of the SQLITE Database in which the table is created.
                                 Defaults to 'default.db'.
    '''

    # Step 1: Setup local logging
    import logging
    logging.basicConfig(level=logging.INFO,
                        format='%(asctime)s %(levelname)s: %(message)s',
                        datefmt='%Y-%m-%d %H:%M:%S')

    # Step 2: Find columns in the dataframe
    cols = input_df.columns
    cols_string = ','.join(cols)
    val_wildcard_string = ','.join(['?'] * len(cols))

    # Step 3: Connect to a DB file if it exists, else crete a new file
    con = sqlite3.connect(db_name)
    cur = con.cursor()
    logging.info(f'SQL DB {db_name} created')

    # Step 4: Create Table
    sql_string = f"""CREATE TABLE {table_name} ({cols_string});"""
    cur.execute(sql_string)
    logging.info(f'SQL Table {table_name} created with {len(cols)} columns')

    # Step 5: Upload the dataframe
    rows_to_upload = input_df.to_dict(orient='split')['data']
    sql_string = f"""INSERT INTO {table_name} ({cols_string}) VALUES ({val_wildcard_string});"""
    cur.executemany(sql_string, rows_to_upload)
    logging.info(f'{len(rows_to_upload)} rows uploaded to {table_name}')

    # Step 6: Commit the changes and close the connection
    con.commit()
    con.close()

In [4]:
def sql_query_to_pd(sql_query_string: str, db_name: str ='default.db') -> pd.DataFrame:
    '''Execute an SQL query and return the results as a pandas dataframe
    Args:
        sql_query_string (str): SQL query string to execute
        db_name (str, optional): Name of the SQLITE Database to execute the query in.
                                 Defaults to 'default.db'.
    Returns:
        pd.DataFrame: Results of the SQL query in a pandas dataframe
    '''
    # Step 1: Connect to the SQL DB
    con = sqlite3.connect(db_name)

    # Step 2: Execute the SQL query
    cursor = con.execute(sql_query_string)

    # Step 3: Fetch the data and column names
    result_data = cursor.fetchall()
    cols = [description[0] for description in cursor.description]

    # Step 4: Close the connection
    con.close()

    # Step 5: Return as a dataframe
    return pd.DataFrame(result_data, columns=cols)

In [5]:
input_df1 = pd.read_csv('departments.csv')

In [6]:
input_df2 = pd.read_csv('dept_emp.csv')

In [7]:
input_df3 = pd.read_csv('dept_manager.csv')

In [8]:
input_df4 = pd.read_csv('employees.csv')

In [9]:
input_df5 = pd.read_csv('salaries.csv')

In [10]:
input_df6 = pd.read_csv('titles.csv')

In [11]:
pd_to_sqlDB(input_df1,
            table_name='departments',
            db_name='default1.db')

In [12]:
pd_to_sqlDB(input_df2,
            table_name='dept_emp',
            db_name='default2.db')

In [13]:
pd_to_sqlDB(input_df3,
            table_name='dept_manager',
            db_name='default3.db')

In [14]:
pd_to_sqlDB(input_df4,
            table_name='employees',
            db_name='default4.db')

In [15]:
pd_to_sqlDB(input_df5,
            table_name='salaries',
            db_name='default5.db')

In [16]:
pd_to_sqlDB(input_df6,
            table_name='titles',
            db_name='default6.db')

In [17]:
sql_query_string1 = """
    SELECT * FROM departments
"""

In [18]:
result_df1 = sql_query_to_pd(sql_query_string1, db_name='default1.db')
result_df1

,dept_no,dept_name
0,d001,Marketing
1,d002,Finance
2,d003,Human Resources
3,d004,Production
4,d005,Development
5,d006,Quality Management
6,d007,Sales
7,d008,Research
8,d009,Customer Service


In [19]:
sql_query_string2 = """
    SELECT * FROM dept_emp
"""

In [20]:
result_df2 = sql_query_to_pd(sql_query_string2, db_name='default2.db')
result_df2

,emp_no,dept_no
0,10001,d005
1,10002,d007
2,10003,d004
3,10004,d004
4,10005,d003
...,...,...
331598,499995,d004
331599,499996,d004
331600,499997,d005
331601,499998,d002


In [21]:
sql_query_string3 = """
    SELECT * FROM dept_manager
"""

In [22]:
result_df3 = sql_query_to_pd(sql_query_string3, db_name='default3.db')
result_df3

,dept_no,emp_no
0,d001,110022
1,d001,110039
2,d002,110085
3,d002,110114
4,d003,110183
5,d003,110228
6,d004,110303
7,d004,110344
8,d004,110386
9,d004,110420


In [23]:
sql_query_string5= """
     SELECT *
     FROM employees
""";

In [36]:
####QUESTION 2 ANSWER:
#result_df4 = sql_query_to_pd(sql_query_string5, db_name='default4.db')
df=pd.read_csv("employees.csv")
df.head()
results=df["hire_date"]=pd.to_datetime(df["hire_date"])
df["year"]=results.dt.strftime("%Y")
answer2=df[df["year"]=="1986"][["first_name","last_name","year"]]
answer2

,first_name,last_name,year
5,Eran,Cusworth,1986
14,Bojan,Zallocco,1986
19,Nevio,Demizu,1986
29,Ziva,Vecchi,1986
33,Mohit,Speek,1986
...,...,...,...
299986,Uriel,Heijenga,1986
299998,Ziyad,Constantine,1986
300006,Yishay,Maksimenko,1986
300007,Yannik,Ranai,1986


In [25]:
sql_query_string5 = """
    SELECT * FROM salaries
"""

In [26]:
result_df5 = sql_query_to_pd(sql_query_string5, db_name='default5.db')
result_df5

,emp_no,salary
0,10001,60117
1,10002,65828
2,10003,40006
3,10004,40054
4,10005,78228
...,...,...
300019,499995,40000
300020,499996,58058
300021,499997,49597
300022,499998,40000


In [27]:
sql_query_string6 = """
    SELECT * FROM titles
"""

In [28]:
result_df6 = sql_query_to_pd(sql_query_string6, db_name='default6.db')
result_df6

,title_id,title
0,s0001,Staff
1,s0002,Senior Staff
2,e0001,Assistant Engineer
3,e0002,Engineer
4,e0003,Senior Engineer
5,e0004,Technique Leader
6,m0001,Manager


In [42]:
###QUESTION 1 ANSWER:
df4subsetq1=result_df4[["emp_no","last_name","first_name","sex"]]
df1=pd.merge(df4subsetq1,result_df5,how="inner",left_on="emp_no",right_on="emp_no")
df1

,emp_no,last_name,first_name,sex,salary
0,473302,Zallocco,Hideyuki,M,40000
1,475053,Delgrande,Byong,F,53422
2,57444,Babb,Berry,F,48973
3,421786,Verhoeff,Xiong,M,40000
4,282238,Baumann,Abdelkader,F,40000
...,...,...,...,...,...
300019,464231,Eastman,Constantino,M,69696
300020,255832,Dayang,Yuping,F,75355
300021,76671,Plessier,Ortrud,M,61886
300022,264920,Samarati,Percy,F,62772


In [30]:
### Sales Department:
df2=pd.merge(result_df3,df1,how="inner",left_on="emp_no",right_on="emp_no")
df3=pd.merge(df2,result_df1,how="inner",left_on="dept_no",right_on="dept_no")
df3[df3["dept_name"]=="Sales"][["last_name","first_name","dept_name"]]

,last_name,first_name,dept_name
16,Kaelbling,Przemyslawa,Sales
17,Zhang,Hauke,Sales


In [31]:
### SALES AND DEVELOPMENT DEPARTMENTS:
df3[(df3["dept_name"]=="Sales")|(df3["dept_name"]=="Development")][["emp_no","last_name","first_name","dept_name"]]

,emp_no,last_name,first_name,dept_name
10,110511,Hagimont,DeForest,Development
11,110567,DasSarma,Leon,Development
16,111035,Kaelbling,Przemyslawa,Sales
17,111133,Zhang,Hauke,Sales


In [32]:
##FREQUENCY COUNTS:
df3["last_name"].value_counts()

Markovitch      1
Minakawa        1
Spinelli        1
Giarratana      1
Butterworth     1
Kambil          1
Staelin         1
Zhang           1
Kaelbling       1
Pesch           1
Quadeer         1
Hofmeyr         1
Onuegbe         1
DasSarma        1
Hagimont        1
Ghazalie        1
Kieras          1
Cools           1
Wegerle         1
Sigstam         1
Ossenbruggen    1
Legleitner      1
Alpin           1
Weedman         1
Name: last_name, dtype: int64